# 用ctypes和c交互

[ctypes](https://docs.python.org/2/library/ctypes.html)是原版python和pypy都支持的一种与c交互的方式,他的原理其实是利用python调用c/c++的动态链接库(就是.so或者.dll那个文件),也就是说c还是那个c,python还是那个python,不用再学其他的了,这也是最推荐的一种与c直接交互的方式.

> 例子:一个二维向量运算的定义

### c代码:

v.c

```c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>

//向量结构体
typedef struct{
    float x,y;
    char repr[50];
}Vector;

Vector add(Vector a,Vector b){
    Vector c;
    c.x = a.x+b.x;
    c.y = a.y+b.y;
    char tempx[10];
    char tempy[10];
    //char x = itoa(c.x,&tempx,10);
    //char y = itoa(c.y,&tempy,10);
    int len = sprintf(c.repr, "<x:%f,y:%f>\n",c.x,c.y);
    //c.repr = "<x:" + x + "," + "y:" + y + ">";
    return c;
}

int main(void){
    Vector a,b,c;
    a.x = 10.0;
    a.y = 20.0;
    strcpy(a.repr,"<x:10.0,y:20.0>");

    b.x = 1.0;
    b.y = 2.0;
    strcpy(b.repr,"<x:10.0,y:20.0>");
    c = add(a,b);
    printf("x %f y %f\n",c.x,c.y);
    printf("%s\n",c.repr);
    return 0;
}
```

编译运行下:

```bash
gcc v.c -o vtest
./vtest
```

结果正常

### 修改成动态链接库(把哪个显示结果的省了)

vector.h

```c
#ifndef VECTOR_HEAD_
#define VECTOR_HEAD_

//向量结构体
typedef struct{
    float x,y;
}Vector;

//向量加法
Vector add(Vector a,Vector b);

#endif
```

vector.c

```c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>

#include "vector.h"

//向量结构体
Vector add(Vector a,Vector b){
    Vector c;
    c.x = a.x+b.x;
    c.y = a.y+b.y;
    return c;
}
```
编译成动态链接库

```bash
gcc -shared -fPIC -o libvec.so vector.c
```

这样就生成一个v.so文件,我们先来用c语言试试看调用

vectorTest.c

```c
#include <stdio.h>
#include <string.h>

#include "vector.h"

int main(void){
    Vector a,b,c;
    a.x = 10.0;
    a.y = 20.0;

    b.x = 1.0;
    b.y = 2.0;
    c = add(a,b);
    printf("x %f y %f\n",c.x,c.y);
    return 0;
}
```

编译:

```bash
gcc vectorTest.c -o vectorTest -L ./ -lvec
./vectorTest
```

结果正常

### 用ctypes调用这个动态链接库

到正题了,我们用这个ctypes调用下动态库试试

In [1]:
from ctypes import CDLL,c_float,c_char_p,Structure,POINTER

In [8]:
class Vector(Structure):
    _fields_ = [("x",c_float),("y",c_float)]
    def __str__(self):
        return "Vector:<{this.x},{this.y}>".format(this=self)
    def __repr__(self):
        return self.__str__()
    def __add__(self,that):
        dll = CDLL("libvec.so")
        dll.add.restype = Vector
        return dll.add(self,that)

In [10]:
v1 = Vector(1,2)

In [11]:
v1

Vector:<1.0,2.0>

In [12]:
v2 = Vector(10,20)

In [13]:
v2

Vector:<10.0,20.0>

In [14]:
v1+v2

Vector:<11.0,22.0>

## ctypes内置数据类型映射:


ctypes type|c type|Python type
---|---|---
c_char|char	|1-character string
c_wchar	|wchar_t	|1-character unicode string
c_byte	|char	|int/long
c_ubyte	|unsigned char	|int/long
c_short	|short	|int/long
c_ushort	|unsigned short	|int/long
c_int	|int	|int/long
c_uint	|unsigned int	|int/long
c_long	|long	|int/long
c_ulong	|unsigned long	|int/long
c_longlong	|__int64 or long long	|int/long
c_ulonglong	|unsigned __int64 or unsigned long long	|int/long
c_float	|float	|float
c_double	|double	|float
c_char_p	|char * (NUL terminated)	|string or None
c_wchar_p	|wchar_t * |(NUL terminated)	|unicode or None
c_void_p	|void *|int/long or None